In [52]:
!vimr 21.txt

In [1]:
data = """789
456
123
.0A"""

data_robot = """.^A
<v>"""


def parse(data):

    direct = {}
    for (y,l) in  enumerate(data.splitlines()):
        for (x,c) in enumerate(l):
            if c == '.':
                continue
            direct[(x,y)] = c
    return tuple((k,v) for k,v in direct.items())
keypad = parse(data)
rob = parse(data_robot)



```
789
456
123
.0A
```

```
 ^A
<v>
```


In [2]:
from heapq import heappush as push, heappop as pop

In [3]:
from functools import cache

In [4]:
dict([(0,1)])

{0: 1}

In [26]:
@cache
def all_path(fmap, start, end):
    map = dict(fmap)
    revmap = {v:k for k,v in map.items()}
    dir= {
        "^":(0,-1),
        "v":(0, 1,),
        ">":(1,0),
        "<":(-1,0),
    }
    open = [(0, start, '')]
    minscore = float('inf')
    paths = []
    k = 0
    while open:
        k+=1
        assert k < 500, (k, fmap, start, end)
        #print(open)
        score, n, seq = pop(open)        
        score = len(seq)
        (x,y) = revmap[n]
        if score > minscore:
            return paths
        if n == end:
            paths.append(seq)
            minscore = len(seq)
            continue
        neigh = [(x+dx, y+dy, d) for d,(dx,dy) in dir.items()]
        #print('N of', start, neigh)
        for (nx,ny, d) in neigh:
            if (nx, ny) in map:
                #print('N ', nx,ny, d,  'in map')
                push(open, (score +1, map[(nx,ny)], seq+d))
            else:
                pass
                #print('N ', nx,ny, d,  'not in map')
                
    return paths
            
            
        

In [27]:
all_path(rob, '<','v')

['>']

In [28]:
from itertools import product
for i,j in  product('v^<>A',repeat=2):
    print(i,j, all_path(rob, i,j))

v v ['']
v ^ ['^']
v < ['<']
v > ['>']
v A ['>^', '^>']
^ v ['v']
^ ^ ['']
^ < ['v<']
^ > ['>v', 'v>']
^ A ['>']
< v ['>']
< ^ ['>^']
< < ['']
< > ['>>']
< A ['>>^', '>^>']
> v ['<']
> ^ ['<^', '^<']
> < ['<<']
> > ['']
> A ['^']
A v ['<v', 'v<']
A ^ ['<']
A < ['<v<', 'v<<']
A > ['v']
A A ['']


In [29]:
all_path(rob, '<', 'v')

['>']

In [30]:
keypad

(((0, 0), '7'),
 ((1, 0), '8'),
 ((2, 0), '9'),
 ((0, 1), '4'),
 ((1, 1), '5'),
 ((2, 1), '6'),
 ((0, 2), '1'),
 ((1, 2), '2'),
 ((2, 2), '3'),
 ((1, 3), '0'),
 ((2, 3), 'A'))

In [31]:
all_path(keypad, '4', 'A')

['>>vv', '>v>v', '>vv>', 'v>>v', 'v>v>']

In [32]:
all_path(rob, 'A', '^')

['<']

In [33]:
from itertools import pairwise

In [102]:
@cache
def shortest(seq, using, depth, maxdepth):
    """
    given a sequence `seq`,  compute the various way the 
    robots can type it, asuming we start at A, and end at A.

    """
    assert depth <= maxdepth, seq
    indent = '  '* depth
    assert not seq.startswith('A')
    assert 'A' not in seq
    acc = ''
    llen = 0
    for source, target in pairwise(iter('A'+seq+'A')):
        if source == target:
            #acc += 'A'
            llen +=1
            
            continue
        all = all_path(using, source, target)
        #print(f"{indent} There are {len(all)} ways to go from {source} to {target} :{all}")
        if not all:
            assert False, (using, source, target)
        #if len(all) == 1:
        #  print(f'{indent} only one way to do {source} -> {target} : {all[0]}')
        #  acc += next(iter(all))+'A'
        if depth == maxdepth:
            mlen = min([len(a) for a in all])
            all = [a for a in all if len(a) == mlen]
            if len(all) == 1:
                delta = next(iter(all))
                #acc += delta+'A'
                llen += len(delta)+1
            else:
                 
                
                #acc += '['+','.join(all)+']'+'A'
                #acc +='_'*len(all[0])+'A'
                llen += len(all[0]) + 1
        else:
            if all:
                possibles = [shortest(a, using,depth+1, maxdepth) for a in all]
                #print(f'{indent} possibilities for {seq} {source} {target}', possibles)
                
                lens = [i for i,s in possibles]
                minlen = min(lens)
                remind= [s for i,s in possibles if i == minlen]
                if len(remind) == 1:
                    #acc += next(iter(remind))
                    llen += minlen
                else:
                    #acc += '(' + ','.join(remind) + ')'
                    llen += minlen
            else:
                assert False
                
            
        #acc +='A'
        #print(f'{indent} ... set acc{depth} way to do {seq} to ', acc)
    return llen, acc
            
    
    

In [103]:


def max_len_input(target, nrob):
    acc = 0
    for source, target in pairwise(iter('A'+target)):
        #print('On keypad:', target)
    
        l3s = []
        for a in all_path(keypad, source, target):
            #l0 = a+'A'
            #print(' Level 1 robot should type', a +'A', 'to type final sequence')
            #l2 = shortest(a, rob, 0, 0)
            #print(' Level 2 robot should type', l2)
            #print()
            
            l3s.append(shortest(a, rob, 0,nrob-1))
    
        ml = min([l for l,_ in l3s])
        mins = [(l,x) for l,x in l3s if l == ml]
        acc += ml
        #print(' Level 3 robot should type', mins)
        #print('---')
    return acc

def score(x, nrob):
    return int(x.replace('A','')) , max_len_input(x, nrob=nrob)        

score('029A', 4)        

(29, 404)

In [104]:
ti = {'029A': '<vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A',
'980A': '<v<A>>^AAAvA^A<vA<AA>>^AvAA<^A>A<v<A>A>^AAAvA<^A>A<vA>^A<A>A',
'179A': '<v<A>>^A<vA<A>>^AAvAA<^A>A<v<A>>^AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A',
'456A': '<v<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^A<A>A<vA>^A<A>A<v<A>A>^AAvA<^A>A',
'379A': '<v<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A',
}

ti = {
'805A':'',
'983A':'',
'149A':'',
'413A':'',
'582A':'',
}


s = 0
for x in ti.keys():
    (a,b)=score(x, 2)
    print(x, a,'*',b , '=', a*b)
    s+=a*b
s # 202648

805A 805 * 72 = 57960
983A 983 * 66 = 64878
149A 149 * 76 = 11324
413A 413 * 70 = 28910
582A 582 * 68 = 39576


202648

In [107]:
s = 0
for x in ti.keys():
    (a,b)=score(x, nrob=25)
    print(x, a,'*',b , '=', a*b)
    s+=a*b
s

805A 805 * 86475783012 = 69613005324660
983A 983 * 80732180764 = 79359733691012
149A 149 * 91059074548 = 13567802107652
413A 413 * 87288844796 = 36050292900748
582A 582 * 86475783008 = 50328905710656


248919739734728

First: 
```
         0    2        9|       A
     <   A| ^ A| >  ^^ A|  vvv  A
  v<<A>>^A|<A>A|vA<^AA>A|<vAAA>^A
```
```
                 0|           2|                   9|                 A
         <       A|       ^   A|     >        ^^   A|        vvv      A
  v <<   A >>  ^ A|   <   A > A|  v  A   <  ^ AA > A|   < v  AAA >  ^ A
<vA<AA>>^AvAA<^A>A|<v<A>>^AvA^A|<vA>^A<v<A>^A>AAvA^A|<v<A>A>^AAAvA<^A>A

```

In [352]:
shortest('<', rob, 0)

(8, '___A___A')

In [139]:
shortest('<', rob, 1)

'([<v,v<]A(<)AA[>>^,>^>]A)A((v)AA[<^,^<]A(>)A)A'

```
< to move the arm from A (its initial position) to 0.
A to push the 0 button.
^A to move the arm to the 2 button and push it.
>^^A to move the arm to the 9 button and push it.
vvvA to move the arm to the A button and push it.
```